# Ejemplo de Sistemas de Ecuaciones

## Planteamiento:

Asumimos que tenemos observaciones de corte transversal independienntes e idénticamente distribuidas:
\begin{equation*}
    \{ \mathbf{X}_i , \mathbf{Y}_i ; i = 1, 2, \ldots, N \}
\end{equation*}

Donde $\mathbf{X}_i$ es una matriz $G \times K$ y $\mathbf{Y}_i$ es un vector $G \times 1$. Así, la matriz $\mathbf{X}_i$ continene todas las variables explicativas que aparecen en el sistema, para cada uno de los individuos indexados $i$:
\begin{equation}
    \mathbf{Y}_i = \mathbf{Y}_i \boldsymbol{\beta} + \boldsymbol{\varepsilon}_i
    \label{Sist_Eqn_G}
\end{equation}

Donde $\boldsymbol{\beta}$ es un vector $K \times 1$ y $\boldsymbol{\varepsilon}_i$ es un vector $G \times 1$. De esta forma en general tendríamos:
\begin{equation*}
    \mathbf{Y}_i = 
    \begin{bmatrix}
    y_{i1} \\
    y_{i2} \\
    \vdots \\
    y_{iG}
    \end{bmatrix}
\end{equation*}

\begin{equation*}
    \boldsymbol{\varepsilon}_i = 
    \begin{bmatrix}
    \varepsilon_{i1} \\
    \varepsilon_{i2} \\
    \vdots \\
    \varepsilon_{iG}
    \end{bmatrix}
\end{equation*}

Donde $i = 1, 2, \ldots, N$. 
\begin{equation*}
    \boldsymbol{\beta} = 
    \begin{bmatrix}
    \boldsymbol{\beta}_{1} \\
    \boldsymbol{\beta}_{2} \\
    \vdots \\
    \boldsymbol{\beta}_{G}
    \end{bmatrix}
\end{equation*}

Par el caso particular de un modelo de ecuaciones simúltneas aparentemente no relacionadas que no compartan variables explicativas:
\begin{equation*}
    \mathbf{x}_{ig} = [x_{i1}, x_{i2}, \ldots, x_{iK_g}]
\end{equation*}

Así, podemos proponer que la matriz $\mathbf{X}_i$ con dimensión $G \times (K_1 + K_2 + \ldots + K_G)$es de la forma:
\begin{equation*}
    \mathbf{X}_i = 
    \begin{bmatrix}
    \mathbf{x}_{i1} \\
    \mathbf{x}_{i2} \\
    \vdots \\
    \mathbf{x}_{iG}
    \end{bmatrix} \\
\end{equation*}


## Estimadores:

El estimador de Mínimos Cuadrados Ordinarios estará dado por:
\begin{eqnarray*}
    \hat{\boldsymbol{\beta}}_{OLS} & = & (\mathbf{X}' \mathbf{X})^{-1} \mathbf{X}' \mathbf{Y}
\end{eqnarray*}

El estimador de Mínimos Cuadrados Generalizados estará dado por:
\begin{eqnarray*}
    \hat{\boldsymbol{\beta}}_{GLS} & = & (\mathbf{X}' \Sigma^{-1} \mathbf{X})^{-1} \mathbf{X}' \Sigma^{-1} \mathbf{Y}
\end{eqnarray*}

## 1. Importación de bibliotecas

In [ ]:
#!pip install linearmodels==4.17
#!pip install linearmodels==4.5

In [ ]:
# Common libraries
#%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels.system import SUR

#
import warnings
warnings.filterwarnings('ignore')

Vamos a utilizar un data set que está en la biblioteca: linearmodels

In [ ]:
from linearmodels.datasets import fringe
print(fringe.DESCR)

In [ ]:
#
fdata = fringe.load()
fdata

Este es un problema de simúltaneidad, en el que tenemos un sistema de ecuaciones:

$Y_1 = a X + b Y_2 + \eta$

$Y_2 = a X + b Y_1 + \nu$

¿cómo proceder en este caso?... Estimamos un modelo en el que asumimos que la información omitida es capturada por un mismo tipo de error:
\begin{equation}
    \begin{bmatrix}
    Y_1 \\
    Y_2
    \end{bmatrix} =
    \begin{bmatrix}
    X_1 \\
    X_2 
    \end{bmatrix} +
    \begin{bmatrix}
    \varepsilon \\
    \varepsilon
    \end{bmatrix}
\end{equation}

In [ ]:
# Seleccionemos nuestras variables exógenas:

exog = ['educ','exper','expersq','tenure','tenuresq','union','south',
        'nrtheast','nrthcen','married','white','male']

X = sm.add_constant( fdata[exog] )

X

## 2. Estimación 

In [ ]:
#!pip install collections

In [ ]:
# Importamos una libreria 
from collections import OrderedDict

fmod_data = OrderedDict()
fmod_data['hrearn'] = {'dependent': fdata.hrearn, 'exog': X}
fmod_data['hrbens'] = {'dependent': fdata.hrbens, 'exog': X}
fmod = SUR(fmod_data)

In [ ]:
print(fmod.fit(cov_type='unadjusted'))

In [ ]:
print(fmod.fit(method='gls', cov_type='unadjusted'))

## 3. ¿Qué pasa cuando los regresores difieren en algunas variables?

\begin{equation}
    \begin{bmatrix}
    Y_1 \\
    Y_2
    \end{bmatrix} =
    \begin{bmatrix}
    X_1 & 0 & Z_1\\
    0 & X_2 & Z_2
    \end{bmatrix} +
    \begin{bmatrix}
    \varepsilon \\
    \varepsilon
    \end{bmatrix}
\end{equation}

In [ ]:
# Definimos dos set de regresores
exog_earn = ['educ','exper','expersq','union','nrtheast','white']
exog_bens = ['educ','exper','expersq','tenure','tenuresq','union','male']

#
X_1 = sm.add_constant(fdata[ exog_earn ])
X_2 = sm.add_constant(fdata[ exog_bens ])

#
fmod_data['hrearn'] = {'dependent': fdata.hrearn, 'exog': X_1}
fmod_data['hrbens'] = {'dependent': fdata.hrbens, 'exog': X_2}
fmod = SUR(fmod_data)

In [ ]:
print(fmod.fit(cov_type='unadjusted'))

In [ ]:
fmod_res= fmod.fit(cov_type='unadjusted', iterate=True)
print(fmod_res)

In [ ]:
fmod_res.iterations

## 4. Hagamos una estimación

ASumimos que:
\begin{eqnarray*}
    \hat{\boldsymbol{\beta}}_{GLS} & = & (\mathbf{X}' \Sigma^{-1} \mathbf{X})^{-1} \mathbf{X}' \Sigma^{-1} \mathbf{Y}
\end{eqnarray*}

y una matriz estimada como:

\begin{eqnarray*}
    \hat{\Sigma} & = &
    \begin{bmatrix}
    e^2_1 & e_1 e_2 & \ldots & e_1 e_n \\
    e_2 e_1 & e^2_2 & \ldots & e_2 e_n \\
    \vdots & \vdots & \ddots & \vdots \\
    e_n e_1 & e_n e_2 & \ldots & e^2_n 
    \end{bmatrix}
\end{eqnarray*}

Donde $e_i = y_i - \mathbf{x}_i \boldsymbol{\beta}$ es el error estimado

In [ ]:
fres_het = fmod.fit(cov_type='robust')
print(fres_het.summary)